# ✨ **Reto Databricks -- Arquitectura Medallion con Datos desde MySQL**
✨

------------------------------------------------------------------------

## 🧠 Descripción general

En este reto los estudiantes deberán construir un **flujo de datos
completo estilo *Medallion Architecture*** (Bronze → Silver → Gold)
dentro de **Databricks**, utilizando como fuente una base de datos
**MySQL**.

El objetivo final será generar un **resumen de ventas mensuales por
cliente y categoría de producto**, aplicando buenas prácticas de
ingeniería de datos con PySpark y Delta Lake.

------------------------------------------------------------------------

## 🎯 Objetivo del reto

Diseñar, desarrollar y documentar un pipeline que: 
1. **Extraiga** datos desde una base MySQL con tablas de tipo transaccional.\
2. **Cargue** los datos crudos en la capa **Bronze** (raw ingestion).\
3. **Transforme y limpie** los datos en la capa **Silver** (refined data).\
4. **Genere un resumen analítico** en la capa **Gold**, con ventas
mensuales agregadas por cliente y categoría de producto.

### Datos de la fuente

- host: www.bigdataybi.com
- port: 3306
- type: Mysql
- user: curso
- password: TuPasswordSeguro123 (Utiliza secretos)
- database: farmafake

------------------------------------------------------------------------

## 🏗️ Tablas origen (en MySQL)

Los datos se encuentran en la base de datos `farmafake` con las siguientes
tablas:

  -----------------------------------------------------------------------
  Tabla                   Descripción
  ----------------------- -----------------------------------------------
  **customers**           Información de clientes: nombre, email,
                          teléfono, género, fecha de nacimiento, número
                          de cliente (cédula o RUC).

  **stores**              Datos de tiendas con latitud, longitud y
                          nombre.

  **products**            Catálogo de productos con categoría, marca,
                          modelo, costo unitario y descuento.

  **invoice_header**      Cabecera de las facturas con cliente, tienda,
                          fecha, totales y estado del documento.

  **invoice_details**     Detalles de los productos facturados por cada
                          `doc_id`.
  -----------------------------------------------------------------------

**NOTA**: Considera que las tablas invoice_header e invoice_details pueden sufrir update, por lo tanto usa MERGE
------------------------------------------------------------------------

## ⚙️ Etapas del desafío

### 🥉 **1. Capa Bronze -- Ingesta cruda**

-   Crear una conexión **JDBC** hacia MySQL (usando secretos en Databricks).\

-   Extraer las cinco tablas (`customers`, `stores`, `products`,`invoice_header`, `invoice_details`).\

-   Guardarlas en formato **Delta** en el catalogo curso y cada esquema debe tener el nombre del participante:

        curso.[participante].bronze_table

-   Cada tabla debe incluir una columna adicional `ingestion_date` con
    la fecha de carga.

💡 *Tip:* Usa `.option("mergeSchema", "true")` y
`.option("overwriteSchema", "true")` para cargas seguras.

------------------------------------------------------------------------

### 🥈 **2. Capa Silver -- Limpieza y estandarización**

A partir de los datos en Bronze: - Eliminar registros duplicados.\
- Estandarizar nombres de columnas a **snake_case**.\
- Convertir los tipos de datos (por ejemplo, fechas y montos
numéricos).\
- Validar que no haya clientes sin `customer_number`, productos sin
`category`, o facturas con `doc_total <= 0`.\
- Crear tablas **Silver** para cada entidad y guardarlas en tu esquema con nombre silver_[table]

💡 *Tip:* Usa `withColumnRenamed` para limpiar nombres y `filter()` para
descartar datos inválidos.

------------------------------------------------------------------------

### 🥇 **3. Capa Gold -- Modelo analítico**

A partir de las tablas Silver: 
1. Unir: 
- `invoice_header` con `invoice_details` por `doc_id`. 
- `customers` por `customer_id`. 
- `products` por `product_id`. 

2. Calcular el monto neto por línea:
```python    
monto = quantity * unit_price * (1 - discount_percent/100)
```

3. Agregar el total por mes, cliente y categoría de producto:

```python
ventas_mensuales = (
  df.groupBy("anio", "mes", "customer_id", "customer_name", "category").agg(F.sum("monto").alias("venta_total"))    
  )
```
4. Guardar el resultado en la capa Gold

💡 *Tip:* Extrae el año y mes con `F.year("doc_date")` y
`F.month("doc_date")`.

------------------------------------------------------------------------

## 📊 Resultado esperado

Una tabla final llamada **`sales_summary`** en la capa **Gold** con las
siguientes columnas:

```sh
  Columna           Descripción
  ----------------- --------------------------------------------------
  `anio`            Año de la venta
  `mes`             Mes de la venta
  `customer_id`     ID del cliente
  `customer_name`   Nombre del cliente
  `category`        Categoría del producto
  `venta_total`     Total vendido en ese mes por cliente y categoría
``` 

Ejemplo de salida: 
```sh
\| anio \| mes \| customer_name \| category \|venta_total \|
\|------\|-----\|----------------\|-----------\|--------------\| 
\| 1 \| Juan Pérez \| Antibióticos \| 1200.50 \| \| 2025 
\| 1 \| María Gómez \| Vitaminas \| 860.90 \| \| 2025

```

------------------------------------------------------------------------

## 🧪 Entregables

1.  **Notebook Databricks** (`challenge_medallion_mysql.dbc` o `.ipynb`)
    con:
    -   Secciones claramente separadas: Bronze / Silver / Gold.
    -   Código PySpark bien documentado.
2.  **Diagrama** simple (opcional) mostrando el flujo de datos.\
3.  **Validación final** mostrando la consulta sobre la tabla
    `sales_summary`.

------------------------------------------------------------------------

## 🏆 Criterios de evaluación

```sh
  Criterio                                    Ponderación
  ------------------------------------------- -------------
  Estructura y limpieza del código            20%
  Correcta ingesta desde MySQL                20%
  Transformaciones y validaciones en Silver   25%
  Resultado correcto en Gold                  25%
  Presentación y explicación del proceso      10%
```

------------------------------------------------------------------------

## 🚀 Bonus Challenge (opcional)

-   Agrega una **columna de fecha de última actualización
    (`update_date`)** en Silver y Gold.\
-   Implementa **carga incremental** desde MySQL usando una columna
    `doc_date`.\
-   Publica la tabla `sales_summary` como **tabla Unity Catalog** para
    consulta desde SQL Warehouse.
